In [246]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy.sparse import hstack, csr_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier



Leitura do CSV

In [226]:
df = pd.read_csv("../dados/dados_transformados.csv")


In [227]:
df.head()

,Titulo,Texto,Origem,url,Classificacao,Editora,Site Editora,Editora_cod,dominio,dominio_cod,Site_Editora_cod,origem_simplificada,origem_cod,conteudo
0,Vídeo com vaias a Nando Reis teve áudio manipu...,“Nando Reis se F… Se F…. no Rock in Rio”,pessoa,https://lupa.uol.com.br/jornalismo/2022/09/09/...,1,Lupa - UOL,lupa.uol.com.br,5,lupa.uol.com.br,2,5,Pessoa,8,Vídeo com vaias a Nando Reis teve áudio manipu...
1,Documentos confirmam uso de dinheiro vivo na c...,"“Não foi dinheiro vivo. Moeda corrente, quer d...",Cristina Graeml,https://lupa.uol.com.br/jornalismo/2022/09/09/...,1,Lupa - UOL,lupa.uol.com.br,5,lupa.uol.com.br,2,5,Outros,7,Documentos confirmam uso de dinheiro vivo na c...
2,G1 não publicou notícia de que rainha Elizabet...,"“Em seus últimos momentos de vida, rainha Eliz...",Imagem no WhatsApp,https://lupa.uol.com.br/jornalismo/2022/09/08/...,1,Lupa - UOL,lupa.uol.com.br,5,lupa.uol.com.br,2,5,WhatsApp,11,G1 não publicou notícia de que rainha Elizabet...
3,TSE não proibiu governo de tomar medidas para ...,“TSE MARCOU PARA PRÓXIMA SEMANA JULGAMENTO OND...,pessoa,https://lupa.uol.com.br/jornalismo/2022/09/08/...,1,Lupa - UOL,lupa.uol.com.br,5,lupa.uol.com.br,2,5,Pessoa,8,TSE não proibiu governo de tomar medidas para ...
4,É falso que órgão de saúde dos EUA passou a re...,"De repente, a Ivermectina aparece no site do N...",Post no WhatsApp,https://lupa.uol.com.br/jornalismo/2022/09/08/...,1,Lupa - UOL,lupa.uol.com.br,5,lupa.uol.com.br,2,5,WhatsApp,11,É falso que órgão de saúde dos EUA passou a re...


Aqui estamos usando o TFidVectorizer para transformar textos em numeros, transformando uma coleção de texto em uma matriz indicando a importancia do peso.

In [228]:
vectorizer = TfidfVectorizer(max_features=10000)
X_texto = vectorizer.fit_transform(df['conteudo'])
df.dtypes

Titulo                 object
Texto                  object
Origem                 object
url                    object
Classificacao           int64
Editora                object
Site Editora           object
Editora_cod             int64
dominio                object
dominio_cod             int64
Site_Editora_cod        int64
origem_simplificada    object
origem_cod              int64
conteudo               object
dtype: object

Aqui estamos juntando as tabelas que serão usadas para o treinamento.

In [229]:
from imblearn.over_sampling import SMOTE
X_outros = df[['dominio_cod', 'Editora_cod', 'Site_Editora_cod', 'origem_cod']].values

X_final = hstack([X_texto, csr_matrix(X_outros)])



Aqui definimos o vetor alvo, e tambem aplicamos o metodo do SMOTE para corrigir o desbalanceamento de dados

In [230]:
y = df['Classificacao']
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_final, y)

Aqui dividimos os dados balanceados em conjunto de treino e teste

In [231]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X_res, y_res, test_size=0.3, stratify=y_res, random_state=42
)



Treinando o Modelo do RandomForest

In [232]:
rf_modelo = RandomForestClassifier(
    n_estimators=100,
    class_weight='balanced',  # ← MUITO IMPORTANTE!
    random_state=42
)
rf_modelo.fit(X_treino, y_treino)
rf_pred = rf_modelo.predict(X_teste)

Treinando Logistica de Regressão

In [233]:
lr_modelo = LogisticRegression(
    max_iter=10000,
    class_weight='balanced',
    solver='liblinear',
    random_state=42
)

lr_modelo.fit(X_treino, y_treino)
lr_pred = lr_modelo.predict(X_teste)

print("Logistica Regressão")
print(classification_report(y_teste, lr_pred))

Logistica Regressão
              precision    recall  f1-score   support

           0       0.84      0.97      0.90      6144
           1       0.96      0.82      0.88      6143

    accuracy                           0.89     12287
   macro avg       0.90      0.89      0.89     12287
weighted avg       0.90      0.89      0.89     12287



Treinando o XGBClassifier

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=(y_res == 0).sum() / (y_res == 1).sum(),
    random_state=42
)
xgb_model.fit(X_treino, y_treino)
xgb_pred = xgb_model.predict(X_teste)
print(classification_report(y_teste, xgb_pred, target_names=["Verdadeiro", "Falso"]))
print(confusion_matrix(y_teste, xgb_pred))

c:\Users\Saulo\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [01:36:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

  Verdadeiro       0.94      0.90      0.92      6144
       Falso       0.91      0.94      0.92      6143

    accuracy                           0.92     12287
   macro avg       0.92      0.92      0.92     12287
weighted avg       0.92      0.92      0.92     12287

[[5538  606]
 [ 343 5800]]


In [ ]:

Aqui pegamos a media da Logistica de Regressão e RandomForest

In [ ]:

rf_acc = accuracy_score(y_teste, rf_pred)
print(f"Acurácia - Random Forest: {rf_acc:.4f}")

lr_acc = accuracy_score(y_teste, lr_pred)
print(f"Acurácia - Logistic Regression: {lr_acc:.4f}")


Acurácia - Random Forest: 0.9469
Acurácia - Logistic Regression: 0.8930


Realizamos a validação cruzada 30x sobre o conjuto balanceado 

In [248]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

xgb_model = XGBClassifier(
    n_estimators=200,
    eval_metric='logloss',
    scale_pos_weight=(y_res == 0).sum() / (y_res == 1).sum(),  # balanceamento
    random_state=42
)

xgb_scores = cross_val_score(xgb_model, X_res, y_res, cv=30, scoring='accuracy')


In [ ]:
rf_modelo = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
rf_scores = cross_val_score(rf_modelo, X_res, y_res, cv=30, scoring='accuracy')



In [236]:

from imblearn.over_sampling import SMOTE
lr_modelo = LogisticRegression(max_iter=10000, solver='liblinear', random_state=42)
lr_scores = cross_val_score(rf_modelo, X_res, y_res, cv=30, scoring='accuracy')




Aqui separamos a media das acuraciais e pegamos o melhor modelo para ser usado na nossa aplicacao

In [ ]:
melhor_modelo = max(
    [('Logistic Regression', lr_modelo, lr_scores.mean()),
     ('Random Forest', rf_modelo, rf_scores.mean()),
     ('XGBoost', xgb_model, xgb_scores.mean())],
    key=lambda x: x[2]
)

modelo_final = melhor_modelo[1]
modelo_final.fit(X_treino, y_treino) 
y_pred = modelo_final.predict(X_teste)

print(f"XGBoost - Média: {xgb_scores.mean():.4f} ± {xgb_scores.std():.4f}")
print(f"Random Forest - Média: {rf_scores.mean():.4f} ± {rf_scores.std():.4f}")
print(f"Logistic Regression - Média: {lr_scores.mean():.4f} ± {lr_scores.std():.4f}")

XGBoost - Média: 0.9113 ± 0.1444
Random Forest - Média: 0.9429 ± 0.0731
Logistic Regression - Média: 0.9429 ± 0.0731


Aqui importamos o modelo fianl e o vectorizer, eu não usei a biblioteca pickle aqui pois estava dando erro de depuração com questões de atualização de biblioteca e outras bibliotecas envolvidas

In [250]:
import joblib

joblib.dump(modelo_final, '../modelo/modelo_treinado.joblib')
joblib.dump(vectorizer, '../modelo/vectorizer_tfidf.joblib')

['../modelo/vectorizer_tfidf.joblib']